#### Setting up
We want to be able to view the tables available and the attributes of the tables. (Your tutor had investigated using freewha.com and the inbuilt MyPHPAdmin but the downside is you can't view the attributes when writing your queries)

This code loads a sql extension for jupyter. Also hides the connected database and other annoying error messages.

We are going to download the Chinook database from github using a command wget, commands which are executed via bash have a ! in front of them.
We are then going to load this database and connect to it.

Adding a sidebar to display useful database stuff.

For future reference if you have a link to another sqlite database you must **Change the url of DB_URL and name of DB_NAME.**

In [1]:
%reload_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False

from ipylab import JupyterFrontEnd, Panel
from ipywidgets import Tab, Text, Output, HTML
app = JupyterFrontEnd()

DB_URL = "https://github.com/lerocha/chinook-database/blob/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite?raw=true"
DB_NAME = "Chinook"
DB_FILE = "Chinook.db"

!wget -q -O $DB_FILE $DB_URL
%sql sqlite:///$DB_FILE

#GET TABLE INFO

tablesList = !sqlite3 $DB_FILE .table
!sqlite3 $DB_FILE .table > tables.txt
tables = [i for line in tablesList for i in line.split()]
tables.sort()
AllTables = " ".join([table for table in tables])


#RIGHT PANEL INFO

right_panel = Panel()
right_panel.title.label = "DB Info"

out = [Output(), Output(), Output()]
with out[0]:
    !./Table_View.sh $DB_FILE $AllTables 
with out[1]:
    !./getAttributes.sh $DB_FILE $AllTables 
with out[2]:
    !sqlite3 $DB_FILE .schema

tab_titles = ['Tables', 'All Table Attributes', 'DDL of Tables']
tab = Tab()
tab.children = out
for i in range(len(tab.children)):
    tab.set_title(i, tab_titles[i])

right_panel.children = [tab]
app.shell.add(right_panel, 'main', { 'mode': 'split-right' })

#ADD ER DIAGRAM
IMG_FILE = f"erd_{DB_NAME}.png"
!eralchemy -i sqlite:///$DB_FILE -o $IMG_FILE

from image_load import load_image
load_image(app, IMG_FILE)


NameError: name 'tablesStr' is not defined

In [ ]:
str(i)

In [4]:
import ipywidgets; ipywidgets.__version__

'7.6.3'

Some Wizardry to Display Table names on the left

In [11]:
help(app.shell)

Help on Shell in module ipylab.shell object:

class Shell(ipywidgets.widgets.widget.Widget)
 |  Shell(*args, **kwargs)
 |  
 |  A parent class for HasTraits that log.
 |  Subclasses have a log trait, and the default behavior
 |  is to get the logger from the currently running Application.
 |  
 |  Method resolution order:
 |      Shell
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add(self, widget, area, args=None)
 |  
 |  expand_left(self)
 |  
 |  expand_right(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from ipywidgets.widgets.widget.Widget:
 |  
 |  __del__(self)
 |      Object disposal
 |  
 |  __init__(self, **kwargs)
 |      Public constructor
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  add_traits(self, **traits)

In [16]:
help(tab)

Help on Tab in module ipywidgets.widgets.widget_selectioncontainer object:

class Tab(_SelectionContainer)
 |  Tab(*args, **kwargs)
 |  
 |  Displays children each on a separate accordion tab.
 |  
 |  Method resolution order:
 |      Tab
 |      _SelectionContainer
 |      ipywidgets.widgets.widget_box.Box
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget_core.CoreWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods inherited from _SelectionContainer:
 |  
 |  get_title(self, index)
 |      Gets the title of a container pages.
 |      
 |      Parameters
 |      ----------
 |      index : int
 |          Index of the container page
 |  
 |  set_title(self, index, title)
 |      Sets the title of a container page.
 |      
 |      Parameters
 |      ----------
 |      index : int
 |        

HTML(value="<style>.--jp_widgets-container-padding { padding' : 15px ; }</style>")

In [3]:
if 'SC' in locals():
    for i in SC:
        i.close()
else:
    SC = [Sidecar(title='Tables'), Sidecar(title='All Table Attributes'), Sidecar(title='DDL of Tables')]
with SC[0]:
    !./Table_View.sh $DB_FILE $AllTables 
with SC[1]:
    !./getAttributes.sh $DB_FILE $AllTables 
with SC[2]:
    !sqlite3 $DB_FILE .schema

To execute sql into this jupyter notebook, we must either make use of the `%sql` or `%%sql` magic command. `%sql` is for a single line and `%%sql` is for an entire cell.

In [4]:
%%sql
SELECT * 
FROM Album
LIMIT 10

AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7
10,Audioslave,8


Alternatively you can access the database using sqlite3 for a sqlite database and execute commands. For this to work; we get a instance of bash working + the commands to go into the sqlite executable. However this is not interactive unlike bash. The developer of the ipython-sql plugin did some work to make it output more prettily

In [10]:
%%bash
sqlite3 Chinook.db
.headers on
.mode markdown
SELECT * FROM Album
LIMIT 10;
.quit

| AlbumId |                 Title                 | ArtistId |
|---------|---------------------------------------|----------|
| 1       | For Those About To Rock We Salute You | 1        |
| 2       | Balls to the Wall                     | 2        |
| 3       | Restless and Wild                     | 2        |
| 4       | Let There Be Rock                     | 1        |
| 5       | Big Ones                              | 3        |
| 6       | Jagged Little Pill                    | 4        |
| 7       | Facelift                              | 5        |
| 8       | Warner 25 Anos                        | 6        |
| 9       | Plays Metallica By Four Cellos        | 7        |
| 10      | Audioslave                            | 8        |
